In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
MAX_TIME_STEP = 30

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df_raw = pd.read_csv("/content/drive/MyDrive/data/archive(1)/LeagueofLegends.csv")
df_raw = df_raw[df_raw['gamelength'] >= MAX_TIME_STEP]
df_raw.reset_index(drop = True, inplace = True)

In [5]:
df = pd.read_csv("/content/drive/MyDrive/data/archive(1)/LeagueofLegends.csv")
df = df[df['gamelength'] >= MAX_TIME_STEP]
df.reset_index(drop = True, inplace = True)
matches = len(df)

In [6]:
def count_item(items):
    count = np.zeros(MAX_TIME_STEP, dtype=np.int8)
    for timestep in range(MAX_TIME_STEP) :
        for item in items:
            if item[0] <= timestep + 1:
                count[timestep] += 1
    return count

In [7]:
from ast import literal_eval
df['golddiff'] = df['golddiff'].apply(literal_eval)
blue = ['bDragons', 'bBarons', 'bHeralds', 'bTowers', 'bInhibs', 'bKills']
red = ['rDragons', 'rBarons', 'rHeralds', 'rTowers', 'rInhibs', 'rKills']
diffs = ['dragondiff', 'barondiff', 'heralddiff', 'towerdiff', 'inhibitordiff', 'killdiff']
for r, b, d in zip(blue, red, diffs):
    df[b] = df[b].apply(literal_eval)
    df[r] = df[r].apply(literal_eval)

    df[b] = df[b].apply(count_item)
    df[r] = df[r].apply(count_item)
    df[d] = df[b] - df[r]

In [8]:
df_raw["bKills"] = df_raw["bKills"].apply(literal_eval)
df_raw["rKills"] = df_raw["rKills"].apply(literal_eval)

In [9]:
bkls = df_raw["bKills"]
btags = df_raw["blueTeamTag"]
bjng = df_raw["blueJungle"]
blue_ganks = []
blue_count = []
for jnglr, res, tag in zip(bjng, bkls, btags):
    #print([(tag +" " + jnglr) in res[i][3] and len(res[i][3]) <= 3 for i in range(len(res))])
    tag = str(tag)
    jnglr = str(jnglr)
    num_ganks = [(tag + " " + jnglr) in res[i][3] and len(res[i][3]) < 2 for i in range(len(res))].count(True)
    blue_ganks.append(num_ganks)
    blue_count.append(len(res))

In [10]:
rkls = df_raw["rKills"]
rtags = df_raw["redTeamTag"]
rjng = df_raw["redJungle"]
red_ganks = []
red_count = []
for jnglr, res, tag in zip(rjng, rkls, rtags):
    #print([(tag +" " + jnglr) in res[i][3] and len(res[i][3]) <= 3 for i in range(len(res))])
    tag = str(tag)
    jnglr = str(jnglr)
    num_ganks = [(tag + " " + jnglr) in res[i][3] and len(res[i][3]) < 2 for i in range(len(res))].count(True)
    red_ganks.append(num_ganks)
    red_count.append(len(res))

In [11]:
df["bGanks"] = blue_ganks
df["rGanks"] = red_ganks
df["bkCount"]= blue_count
df["rkCount"]= red_count

In [12]:
stats = ['golddiff','dragondiff', 'barondiff', 'heralddiff', 'towerdiff', 'inhibitordiff', 'killdiff']
x = df[stats]
y = df['bResult']

In [13]:
from sklearn.preprocessing import StandardScaler

data = {}
scalers = {}
for stat in stats:
    scalers[stat] = StandardScaler()
    for row in df[stat]:
        scalers[stat].partial_fit(np.asanyarray(row).reshape(-1, 1))
    data[stat] = [scalers[stat].transform(np.asanyarray(row).reshape(-1, 1)).reshape(-1) for row in df[stat]]

num_features = len(data)
print(f'# of features per timestep: {num_features}')

# of features per timestep: 7


In [14]:
from keras.models import Sequential
from keras.layers import Dense

In [16]:
model = Sequential([
    Dense(32, activation='relu', input_shape=(7,)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid'),
])

In [17]:
model.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])